# 03_05_morescipy.ipynb - Interpolation and optimization with SciPy

In [ ]:
import numpy as np
import matplotlib.pyplot as pp

import numba

import astropy.constants
import astropy.time
import astropy.coordinates

import scipy.integrate
import scipy.interpolate
import scipy.optimize

#### Solar System integrator from 03_04

In [ ]:
bodies = ['sun','mercury','venus','earth','mars','jupiter','saturn','uranus','neptune']

massdict = {'sun': 1.0,
            'mercury': 1.6601209949637026e-07,
            'venus': 2.4478382857373332e-06,
            'earth': 3.0034896946063695e-06,
            'mars': 3.227156037857755e-07,
            'jupiter': 0.0009547918983127075,
            'saturn': 0.00028588567008942334,
            'uranus': 4.3662495719438076e-05,
            'neptune': 5.151383713179197e-05}

masses = np.array([massdict[body] for body in bodies])

In [ ]:
G = astropy.constants.G.to('AU^3 / (Msun d^2)').value

In [ ]:
@numba.jit
def ydot(t, y):
    # how many bodies? make sure the answer is an integer
    n = int(y.shape[0] / 6)

    # make an empty container for the derivatives
    yd = np.zeros_like(y)
    
    # for each body
    for i in range(n):
        # set x_i' = v_i (array slice assignment)
        yd[i*6:i*6+3] = y[i*6+3:i*6+6]
        
        # loop over all other bodies
        for j in range(n):
            if i == j:
                continue

            # add contribution of planet j to v_i'
            rij = y[j*6:j*6+3] - y[i*6:i*6+3]
            yd[i*6+3:i*6+6] += G * masses[j] * rij / np.dot(rij,rij)**1.5
    
    return yd

In [ ]:
def get_posvel(body, t):
    posvel = astropy.coordinates.get_body_barycentric_posvel(body, t)
    
    return np.hstack([posvel[0].xyz.value.T, posvel[1].xyz.value.T])

In [ ]:
t0, t1 = astropy.time.Time('2021-07-04'), astropy.time.Time('2031-07-04')

In [ ]:
y0 = np.array([get_posvel(body, t0) for body in bodies]).flatten()

In [ ]:
orbits = scipy.integrate.solve_ivp(ydot, [t0.mjd, t1.mjd], y0, rtol=1e-9, atol=1e-9)

#### Interpolating distances

In [ ]:
orbits

In [ ]:
pp.plot(np.diff(orbits.t), '.')

In [ ]:
orbint = scipy.interpolate.interp1d(orbits.t, orbits.y, kind='quadratic')

In [ ]:
orbint(t0.mjd + 365.25)

In [ ]:
ts = astropy.time.Time('2025-01-01').mjd + np.arange(0, 5*365)
oneyear = orbint(ts)[3*6:4*6,:]

for i in range(3):
    pp.plot(ts, orbint(ts)[3*6+i,:], '-')

#### Finding minima

In [ ]:
def get_dist(t, body1, body2, orbint):
    # get position of all bodies at time t
    y = orbint(t)
    
    # compute indices of each body
    i, j = bodies.index(body1), bodies.index(body2)
    
    # compute Euclidian distance
    return np.sqrt(np.sum((y[i*6:i*6+3] - y[j*6:j*6+3])**2, axis=0))

In [ ]:
pp.plot(ts, get_dist(ts, 'jupiter', 'sun', orbint))

In [ ]:
minimum = so.minimize(lambda t: -get_dist(t, 'jupiter', 'sun', orbint),
                      x0=ts[900], bounds=[(ts[0],ts[-1])])

In [ ]:
minimum

In [ ]:
pp.plot(ts, get_dist(ts, 'jupiter', 'sun', orbint))
pp.plot(minimum.x, -minimum.fun, 'ro')

In [ ]:
pp.plot(ts, get_dist(ts, 'mars', 'venus', orbint))

In [ ]:
minimum = so.minimize(get_dist,
                      x0=ts[900], args=('mars','venus',orbint),
                      bounds=[(ts[0],ts[-1])])

In [ ]:
pp.plot(ts, get_dist(ts, 'mars', 'venus', orbint))
pp.plot(minimum.x, minimum.fun, 'ro')

In [ ]:
for x0 in [61000,61750,62500]:
    minimum = so.minimize(get_dist, x0=x0, args=('mars','venus',orbint), bounds=[(ts[0],ts[-1])])
    print(minimum.x, minimum.fun)